# Feedforward NN Model

## Setup

In [1]:
import numpy as np
import torch
import torch.utils.data
import torch.nn as nn
from tensorboardX import SummaryWriter

data = np.load('./dataset/preprocessed.npz')
train_x = torch.from_numpy(data['train_x']).float()
train_y = torch.from_numpy(data['train_y']).float()
test_x = torch.from_numpy(data['test_x']).float()

train_len = int(len(train_x) * 0.8)
eval_len = len(train_x) - train_len
whole_train_data = torch.utils.data.TensorDataset(train_x, train_y)
train_data, eval_data = torch.utils.data.random_split(whole_train_data, [train_len, eval_len])
test_data = torch.utils.data.TensorDataset(test_x)

print('train_x shape:', train_x.shape)
print('train_y shape:', train_y.shape)
print('test_x shape:', test_x.shape)

train_x shape: torch.Size([3000, 20083])
train_y shape: torch.Size([3000])
test_x shape: torch.Size([4398, 20083])


## Define Model

In [2]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(in_features=20082, out_features=8192),
            nn.SELU(),
            nn.Dropout(p=0.5),
            nn.Linear(in_features=8192, out_features=4096),
            nn.SELU(),
            nn.Dropout(p=0.5),
            nn.Linear(in_features=4096, out_features=2048),
            nn.SELU(),
            nn.Dropout(p=0.5),
            nn.Linear(in_features=2048, out_features=1024),
            nn.SELU(),
            nn.Dropout(p=0.5),
            nn.Linear(in_features=1024, out_features=1),
            nn.SELU()
        )
        self.loss_func = nn.MSELoss()
        
    def forward(self, in_data):
        in_data[torch.isnan(in_data)] = 0 # set nan to 0
        pred = self.net(in_data[:,1:]) # skip id
        return pred

    def loss(self, pred, truth):
        loss = torch.sqrt(self.loss_func(torch.log1p(torch.clamp(pred, min=0.0)), torch.log1p(truth))) #RMSLE
        return loss

## Runner function

In [3]:
def run(model, loaders, optimizer, writer, num_epoch=10, device='cpu'):
    def run_epoch(mode):
        epoch_loss = 0.0
        for i, batch in enumerate(loaders[mode], 0):
            in_data, truth = batch
            in_data, truth = in_data.to(device), truth.to(device)

            if mode == 'train':
                optimizer.zero_grad()

            pred = model(in_data)
            batch_loss = model.loss(pred, truth)
            
            epoch_loss += batch_loss.item()
            if mode == 'train':
                batch_loss.backward()
                optimizer.step()
            
            del in_data
            del truth
            torch.cuda.empty_cache()

        # sum of all batchs / num of batches
        epoch_loss /= i + 1
        print('epoch %d %s loss %.4f' % (epoch, mode, epoch_loss))
        # log to tensorboard
        if not (writer is None):
            writer.add_scalars('%s_loss' % model.__class__.__name__,
                         tag_scalar_dict={mode: epoch_loss}, 
                         global_step=epoch)
    for epoch in range(num_epoch):
        if 'train' in loaders:
            model.train()
            run_epoch('train')
        if 'eval' in loaders:
            model.eval()
            run_epoch('eval')

## Training and Validation

In [8]:
model = Model().cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=2.5e-4, betas=(0.9, 0.99))
run(
    model=model,
    loaders={
        'train': torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True),
        'eval': torch.utils.data.DataLoader(eval_data, batch_size=32, shuffle=True)
    },
    optimizer=optimizer, 
    writer=SummaryWriter('./logs/'), 
    num_epoch=15, 
    device='cuda'
)
del model
torch.cuda.empty_cache()

epoch 0 train loss 5.4590
epoch 0 eval loss 4.2746
epoch 1 train loss 3.8986
epoch 1 eval loss 3.6428
epoch 2 train loss 3.5222
epoch 2 eval loss 3.4047
epoch 3 train loss 3.3728
epoch 3 eval loss 3.3508
epoch 4 train loss 3.3112
epoch 4 eval loss 3.2802
epoch 5 train loss 3.2986
epoch 5 eval loss 3.2920
epoch 6 train loss 3.2866
epoch 6 eval loss 3.2758
epoch 7 train loss 3.2738
epoch 7 eval loss 3.2884
epoch 8 train loss 3.2308
epoch 8 eval loss 3.3078
epoch 9 train loss 3.2883
epoch 9 eval loss 3.2883
epoch 10 train loss 3.2716
epoch 10 eval loss 3.2978
epoch 11 train loss 3.2592
epoch 11 eval loss 3.2809
epoch 12 train loss 3.2709
epoch 12 eval loss 3.2773
epoch 13 train loss 3.2353
epoch 13 eval loss 3.2929
epoch 14 train loss 3.2748
epoch 14 eval loss 3.2598


## Train final model over whole dataset and Export test output

In [9]:
model = Model().cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=2e-4, betas=(0.9, 0.99))
run(
    model=model,
    loaders={
        'train': torch.utils.data.DataLoader(whole_train_data, batch_size=32, shuffle=True)
    },
    optimizer=optimizer, 
    writer=SummaryWriter('./logs/final/'), 
    num_epoch=30, 
    device='cuda'
)

model.eval()
test_pred_tensor = model(test_x.to('cuda'))
test_pred = test_pred_tensor.detach().cpu().numpy()
output = np.concatenate((np.expand_dims(data['test_x'][:,0], axis=1), test_pred), axis=1)
np.savetxt('./test_out_fnn.csv', output, header='id,revenue', delimiter=',', fmt='%i', comments='')
print('file saved')

epoch 0 train loss 5.4525
epoch 1 train loss 3.8975
epoch 2 train loss 3.5295
epoch 3 train loss 3.3565
epoch 4 train loss 3.3179
epoch 5 train loss 3.2972
epoch 6 train loss 3.2817
epoch 7 train loss 3.2904
epoch 8 train loss 3.2773
epoch 9 train loss 3.2905
epoch 10 train loss 3.2752
epoch 11 train loss 3.2703
epoch 12 train loss 3.2880
epoch 13 train loss 3.2594
epoch 14 train loss 3.2652
epoch 15 train loss 3.2842
epoch 16 train loss 3.2726
epoch 17 train loss 3.2807
epoch 18 train loss 3.2529
epoch 19 train loss 3.2527
epoch 20 train loss 3.2655
epoch 21 train loss 3.2736
epoch 22 train loss 3.2644
epoch 23 train loss 3.2514
epoch 24 train loss 3.2642
epoch 25 train loss 3.2517
epoch 26 train loss 3.2497
epoch 27 train loss 3.2296
epoch 28 train loss 3.2638
epoch 29 train loss 3.2322
file saved
